In [ ]:
llm_config = {
    'model': 'Meta-Llama-3-8B-Instruct',
    'base_url': 'http://127.0.0.1:7777/v1',
    'api_type': 'open_ai',
    'api_key': 'NULL'
}

In [ ]:
from autogen.coding import LocalCommandLineCodeExecutor

code_execution_config_list = [
    {  # 没有保存临时代码
        "work_dir": "output",
        "use_docker": False
    },
    {  # 可以保存临时代码
        "executor": LocalCommandLineCodeExecutor(work_dir="output")
    }
]

code_execution_config = code_execution_config_list[1] # 可以尝试使用不同的code_execution设置

In [ ]:
from autogen import AssistantAgent, UserProxyAgent

# create an AssistantAgent instance named "assistant" with the LLM configuration.
assistant = AssistantAgent(name="assistant",
                           llm_config=llm_config)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = UserProxyAgent(name="user_proxy",
                            human_input_mode="NEVER",
                            max_consecutive_auto_reply=10,
                            is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
                            code_execution_config=code_execution_config,
                            llm_config=llm_config
                           )

In [ ]:
def my_message_generator():
    # your CSV file
    file_name = "PastHires.csv"
    try:
        with open(file_name, mode="r", encoding="utf-8") as file:
            file_content = file.read()
    except FileNotFoundError:
        file_content = "No data found."
    return "Data: \n" + file_content

task = """
Train a binary-classification model to predict whether a candidate will be hired or not.
after train the model success, save the trained model in a new directory named model-{time} and {time} is a placeholder. give the accuracy metric for the trained model by a validation dataset.
The data file is PastHires.csv, if you cannot find it, try ../PastHires.csv. The label column is Hired.

Important Notes:
- Do not issue commands for 'pip install' or 'pip uninstall'. If there is a dependency missing, proactively terminate the session and instruct the user to handle the dependency installation manually.
- Ensure that complete code is always provided! even if parts of it have been provided before! For example, code cannot includes phrases such as "rest of the code remains the same"!
- Ensure that each response generates at most one code block!
""" + my_message_generator()

In [ ]:
# the assistant receives a message from the user, which contains the task description
chat_res = user_proxy.initiate_chat(
    assistant,
    message=task,
    summary_method="reflection_with_llm"
)

In [ ]:
chat_res.summary